In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from Config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("cities.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hithadhoo,-0.6000,73.0833,80.78,66,0,3.06,MV,2022-09-16 00:45:21
1,1,Mataura,-46.1927,168.8643,46.47,52,92,4.90,NZ,2022-09-16 00:45:21
2,2,Bredasdorp,-34.5322,20.0403,53.42,92,18,3.65,ZA,2022-09-16 00:45:22
3,3,Ushuaia,-54.8000,-68.3000,35.26,87,40,26.46,AR,2022-09-16 00:45:23
4,4,Solnechnyy,50.7214,136.6319,57.40,51,99,2.53,RU,2022-09-16 00:45:23


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hithadhoo,-0.6000,73.0833,80.78,66,0,3.06,MV,2022-09-16 00:45:21
5,5,Miri,4.4148,114.0089,78.94,89,40,6.91,MY,2022-09-16 00:40:34
6,6,Arraial Do Cabo,-22.9661,-42.0278,66.33,91,100,10.27,BR,2022-09-16 00:45:24
8,8,Buraydah,26.3260,43.9750,86.45,20,0,6.69,SA,2022-09-16 00:45:25
10,10,Atuona,-9.8000,-139.0333,76.91,68,1,17.05,PF,2022-09-16 00:45:25
11,11,Mazatlan,23.2167,-106.4167,88.02,84,20,10.36,MX,2022-09-16 00:40:39
12,12,Almeirim,-1.5233,-52.5817,81.48,60,82,6.91,BR,2022-09-16 00:45:26
14,14,Port Alfred,-33.5906,26.8910,60.19,97,96,5.44,ZA,2022-09-16 00:45:27
15,15,Olinala,17.8333,-98.8500,66.49,90,100,2.13,MX,2022-09-16 00:45:27
16,16,Nancha,47.1333,129.2667,64.94,67,100,2.73,CN,2022-09-16 00:45:28


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       465
City          465
Lat           465
Lng           465
Max Temp      465
Humidity      465
Cloudiness    465
Wind Speed    465
Country       463
Date          465
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hithadhoo,-0.6000,73.0833,80.78,66,0,3.06,MV,2022-09-16 00:45:21
5,5,Miri,4.4148,114.0089,78.94,89,40,6.91,MY,2022-09-16 00:40:34
6,6,Arraial Do Cabo,-22.9661,-42.0278,66.33,91,100,10.27,BR,2022-09-16 00:45:24
8,8,Buraydah,26.3260,43.9750,86.45,20,0,6.69,SA,2022-09-16 00:45:25
10,10,Atuona,-9.8000,-139.0333,76.91,68,1,17.05,PF,2022-09-16 00:45:25
11,11,Mazatlan,23.2167,-106.4167,88.02,84,20,10.36,MX,2022-09-16 00:40:39
12,12,Almeirim,-1.5233,-52.5817,81.48,60,82,6.91,BR,2022-09-16 00:45:26
14,14,Port Alfred,-33.5906,26.8910,60.19,97,96,5.44,ZA,2022-09-16 00:45:27
15,15,Olinala,17.8333,-98.8500,66.49,90,100,2.13,MX,2022-09-16 00:45:27
16,16,Nancha,47.1333,129.2667,64.94,67,100,2.73,CN,2022-09-16 00:45:28


In [7]:
#5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy() 
hotel_df["Hotel Name"] = "" 
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hithadhoo,MV,80.78,-0.6000,73.0833,
5,Miri,MY,78.94,4.4148,114.0089,
6,Arraial Do Cabo,BR,66.33,-22.9661,-42.0278,
8,Buraydah,SA,86.45,26.3260,43.9750,
10,Atuona,PF,76.91,-9.8000,-139.0333,
11,Mazatlan,MX,88.02,23.2167,-106.4167,
12,Almeirim,BR,81.48,-1.5233,-52.5817,
14,Port Alfred,ZA,60.19,-33.5906,26.8910,
15,Olinala,MX,66.49,17.8333,-98.8500,
16,Nancha,CN,64.94,47.1333,129.2667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
        hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
hotel_df = hotel_df.dropna()
hotel_df['Hotel Name'].value_counts()

Scoop Guest House                        1
Mega Hotel, Miri                         1
Northland Motor Hotel                    1
Finch Bay Galapagos Hotel                1
COMFORT INN                              1
Courtyard by Marriott Danbury            1
InterContinental Riyadh, an IHG Hotel    1
Aldea del Bazar Hotel and Spa            1
MVC Transient Room                       1
Paradise Inn                             1
Casa Das Faias                           1
Big Trees Mobile Home Community          1
Communications Hotel                     1
The Halyards Hotel                       1
Ricácio Hotel                            1
Hotel La Siesta                          1
Hôtel Hiva Oa Hanakee Lodge              1
Mövenpick Qassim                         1
Pousada Porto Praia                      1
Black Rock Oceanfront Resort             1
Name: Hotel Name, dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
df=hotel_df
hotel_df.to_csv(output_data_file, index_label="City_ID")
df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Hithadhoo,MV,80.78,-0.6000,73.0833,Scoop Guest House
5,Miri,MY,78.94,4.4148,114.0089,"Mega Hotel, Miri"
6,Arraial Do Cabo,BR,66.33,-22.9661,-42.0278,Pousada Porto Praia
8,Buraydah,SA,86.45,26.3260,43.9750,Mövenpick Qassim
10,Atuona,PF,76.91,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
11,Mazatlan,MX,88.02,23.2167,-106.4167,Hotel La Siesta
12,Almeirim,BR,81.48,-1.5233,-52.5817,Ricácio Hotel
14,Port Alfred,ZA,60.19,-33.5906,26.8910,The Halyards Hotel
16,Nancha,CN,64.94,47.1333,129.2667,Communications Hotel
17,Garden City,US,68.25,42.3256,-83.3310,Big Trees Mobile Home Community


In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
hotel_df = pd.read_csv(output_data_file) #this is because i had to debug an issue
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd> </dd>and<dd>{Max Temp} °F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
figs = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
figs.add_layer(heat_layer)
figs.add_layer(marker_layer)

In [14]:
# Call the figure to plot the data.
figs

Figure(layout=FigureLayout(height='420px'))